<a href="https://colab.research.google.com/github/snapdotNG/LSTMCandleDirection/blob/main/LSTM_Prediction_Candle_Direction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from keras.layers import Dense, CuDNNLSTM, Dropout, Bidirectional
from keras.models import Sequential
from tensorflow.keras.optimizers import Nadam, SGD
from sklearn.preprocessing import MinMaxScaler
from binance import Client
from tensorflow.keras.callbacks import EarlyStopping
import time
import telegram_send
from datetime import datetime

model_dir = "/content/drive/MyDrive/BTCLSTM_DATA/Binary"

def handle_telegram():
    telegram_token = '2016117110:AAGJMHU-3GD-jXXXXXXXXXXXXkaU'
    chat_id = '-XXXXXXXXXXXX'
    path_config = telegram_send.get_config_path()

    with open(path_config, 'w') as f:
        f.write(f'[telegram]\ntoken = {telegram_token}\nchat_id = {chat_id}')



def get_binance():
    api_key = "YourAPiKey"
    api = "YourAPiSecret2"
    _client = Client(api_key, api)
    return _client


def pre_processing(win_size):
    temp = get_binance().get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_5MINUTE, "5000 minutes ago UTC")
    temp = temp[-250:]

    all_close_values = []
    all_col_values = []
    scalar = MinMaxScaler(feature_range=(0, 1))

    for row in temp:
        all_close_values.append([float(row[4])])
        all_col_values.append([float(row[1]), float(row[2]), float(row[3]), float(row[4])])

    x_train, y_train = [], []
    scalar.fit(all_col_values)

    for cut_index in range(win_size, len(all_col_values)):
        temp = scalar.transform(all_col_values[cut_index - win_size:cut_index])
        x_train.append(temp[:len(temp)])


        if all_col_values[cut_index][3] > all_col_values[cut_index - 1][3]:
            y_train.append(1)
        else:
            y_train.append(0)

    x_train = np.stack(x_train, axis=0)
    y_train = np.array(y_train, dtype=object)


    return x_train, y_train, scalar, x_train[:10], y_train[:10]


def get_vanilla_model_0(win_size, n_features):
    model = Sequential()
    model.add(CuDNNLSTM(units=50, input_shape=(win_size, n_features), return_sequences=True))
    # model.add(CuDNNLSTM(units=50, return_sequences=True))
    # model.add(CuDNNLSTM(units=50, return_sequences=True))
    # model.add(CuDNNLSTM(units=50, return_sequences=True))
    model.add(CuDNNLSTM(units=50))
    # model.add(Dense(units=50))
    # model.add(Dense(units=50, activation='relu'))
    # model.add(Dense(units=10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mae', 'accuracy'])
    return model


def get_current_processed(sc):
    temp = get_binance().get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_5MINUTE, "4000 minutes ago UTC")
    temp = temp[-21:]
    temp = temp[:20]
    _last_close_time = datetime.fromtimestamp(temp[len(temp) - 2][6] / 1000.0)

    all_close_values = []
    all_col_values = []

    for row in temp:
        all_close_values.append([float(row[4])])
        all_col_values.append([float(row[1]), float(row[2]), float(row[3]), float(row[4])])

    x_train, y_train = [], []

    x_train.append(sc.transform(all_col_values))

    x_train = np.stack(x_train, axis=0)
    x_train = np.asarray(x_train).astype('float32')

    
    return x_train, _last_close_time, all_close_values[len(all_close_values) - 1]

accurate = 0
ctr = 0

def test_model(model, scalar):
    last_prediction = -1
    last_value = None

    global accurate
    global ctr
    
    int_ct = 0
    while True:
        x_train, last_close_time, last_close_value = get_current_processed(scalar)

        prediction = model.predict(x_train)

        final_prediction = 1 if prediction[0][0] > 0.8 else 0

        print("\nStart")
        print("LSTM prediction: {}".format(prediction[0][0]))
        print("Final prediction: {}".format(final_prediction))
        print("Last close: {}".format(last_close_value))

        if last_value:
            if last_prediction == 0:
                if last_value > last_close_value[0]:
                    accurate += 1
            else:
                if last_value < last_close_value[0]:
                    accurate += 1

            print("Model accuracy: {}".format((accurate / ctr) * 100))
            telegram_send.send(messages=["Model average accuracy: {} for: {} predictions".format((accurate / ctr) * 100, ctr - 1)])

        if final_prediction == 1:

            print("Next close will be High")
            last_prediction = 1

        else:
            print("Next close will be Down")
            last_prediction = 0

        last_value = last_close_value[0]
        ctr += 1
        int_ct += 1
        print("Counter: {}".format(ctr))
        print("End\n")

        telegram_send.send(messages=['Last close time: {}\nLast close value: {}\nNext Close Prediction: {}\nWith confidence: {}'.format(last_close_time, last_close_value, "High" if final_prediction == 1 else "Down", round(((0.8 - prediction[0][0]) / 0.8) * 100, 2) if final_prediction == 0 else round((prediction[0][0] / 1) * 100, 2))])

        if int_ct == 2:
            break

        time.sleep(300)
        # break


def train_all_models():
    window_size = 20
    x_train, y_train, sc, x_val, y_val = pre_processing(window_size)
    eph = 1000
    n_features = 4
    bs = 20

    x_train = np.asarray(x_train).astype('float32')
    y_train = np.asarray(y_train).astype('float32')
    x_val = np.asarray(x_val).astype('float32')
    y_val = np.asarray(y_val).astype('float32')

    model_0 = get_vanilla_model_0(window_size, n_features=n_features)

    # es = EarlyStopping(monitor='accuracy', mode='max', min_delta=0.001, verbose=1, patience=50)
    model_0.fit(x_train, y_train, epochs=eph, batch_size=bs, verbose=2)

    test_model(model_0, sc)

    time.sleep(5)
    train_all_models()


if __name__ == '__main__':
    handle_telegram()
    train_all_models()


Streaming output truncated to the last 5000 lines.
12/12 - 0s - loss: 0.0750 - mae: 0.0624 - accuracy: 0.9913
Epoch 525/1000
12/12 - 0s - loss: 0.0709 - mae: 0.0590 - accuracy: 0.9826
Epoch 526/1000
12/12 - 0s - loss: 0.0815 - mae: 0.0661 - accuracy: 0.9826
Epoch 527/1000
12/12 - 0s - loss: 0.0827 - mae: 0.0642 - accuracy: 0.9826
Epoch 528/1000
12/12 - 0s - loss: 0.0786 - mae: 0.0654 - accuracy: 0.9826
Epoch 529/1000
12/12 - 0s - loss: 0.0796 - mae: 0.0662 - accuracy: 0.9913
Epoch 530/1000
12/12 - 0s - loss: 0.0808 - mae: 0.0624 - accuracy: 0.9783
Epoch 531/1000
12/12 - 0s - loss: 0.1022 - mae: 0.0755 - accuracy: 0.9652
Epoch 532/1000
12/12 - 0s - loss: 0.0838 - mae: 0.0686 - accuracy: 0.9783
Epoch 533/1000
12/12 - 0s - loss: 0.0900 - mae: 0.0696 - accuracy: 0.9739
Epoch 534/1000
12/12 - 0s - loss: 0.0765 - mae: 0.0636 - accuracy: 0.9870
Epoch 535/1000
12/12 - 0s - loss: 0.0679 - mae: 0.0562 - accuracy: 0.9783
Epoch 536/1000
12/12 - 0s - loss: 0.0620 - mae: 0.0522 - accuracy: 0.9913
Ep

In [ ]:
!pip install telegram-send
!pip install python-binance

     |████████████████████████████████| 490 kB 6.1 MB/s 
     |████████████████████████████████| 58 kB 8.4 MB/s 
     |████████████████████████████████| 428 kB 84.6 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: cachetools
    Found existing installation: cachetools 4.2.4
    Uninstalling cachetools-4.2.4:
      Successfully uninstalled cachetools-4.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires tornado~=5.1.0; python_version >= "3.0", but you have tornado 6.1 which is incompatible.


     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 214 kB 8.4 MB/s 
     |████████████████████████████████| 288 kB 80.5 MB/s 
     |████████████████████████████████| 103 kB 68.3 MB/s 
     |████████████████████████████████| 1.3 MB 48.4 MB/s 
     |████████████████████████████████| 271 kB 74.0 MB/s 
     |████████████████████████████████| 160 kB 80.8 MB/s 
